In [1]:
import sys
home = '/home/nina/CopulaGP/'
sys.path.insert(0, home)

import torch
import numpy as np
from scipy.stats import t

import utils
from benchmarks import train4entropy, integrate_student
import bvcopula
from vine import CVine

# Student

In [2]:
#choose the number of dimensions
Nvar = 2
mc_size = 2000
sem_tol=0.015

NSamp=10000
device = torch.device('cuda:0')
x = torch.linspace(0.,1.,NSamp).numpy()
train_x = torch.tensor(x).float().to(device=device)

#define functions
Frhos = lambda NN: np.ones(NN)*0.7
Fdfs = lambda NN: np.exp(5*np.linspace(0,1,NN))+1
rhos = Frhos(NSamp)
dfs = Fdfs(NSamp)

In [3]:
# sample from Student
y = utils.student_rvs(Nvar,rhos,dfs,1).squeeze()
y0 = np.zeros_like(y)
for i in range(y.shape[0]):
    y0[i] = t.cdf(y[i],df=dfs[i])
train_y = torch.tensor(y0).float().to(device=device)

In [4]:
# define pre-selected likelihoods
likelihoods =  [bvcopula.GaussianCopula_Likelihood(),
                bvcopula.GumbelCopula_Likelihood(rotation='180°')] 
waic, model = bvcopula.infer(likelihoods,train_x,train_y,device=device)

In [5]:
model.gp_model.eval()
copula = model.marginalize(train_x)

In [6]:
def get_R(model,copula):
    ints = np.array([0,1])
    R = []
    N, M = 20, 100
    mesh = torch.stack(torch.meshgrid([torch.linspace(0.,1.,M+2)[1:-1], 
                                       torch.linspace(0.,1.,N+2)[1:-1]]),axis=-1).reshape(-1,2).to(device)
    for i in range(len(ints)-1):
        mask = (x>ints[i]) & (x<ints[i+1])
        S = y0[mask]
        u_cdf = np.cumsum(np.histogram2d(*S.T,bins=[M+1,N])[0],axis=0)
        ecdf = u_cdf[:-1] / u_cdf.max(axis=0)
        # plt.imshow(ecdf)
        mean = x[mask].mean()
        test_x = (torch.ones(M*N)*mean).float().cuda(device=device)
        with torch.no_grad():
            f = model.gp_model(test_x).mean
            copula = model.likelihood.get_copula(f)
        ccdf = copula.ccdf(mesh).detach().cpu().numpy().reshape(M,N)
        # plt.imshow(ccdf)
        # base = np.repeat((np.linspace(0,1,N+2)[1:-1]).reshape(-1,1),N,axis=1)
        var_exp = 1 - ((ecdf-ccdf)**2).sum(axis=0)/((ecdf-0.5)**2).sum(axis=0)
        R.append(var_exp.mean())
    return R
get_R(model,copula)

[0.9907808368077073]

# Transformed Gauss

In [7]:
rho0 = torch.linspace(-0.1,.999,NSamp,device=device).unsqueeze(0)
def const_rho_layers(rho,Nvar):
    def lin_gauss(rhos):
        return bvcopula.MixtureCopula(rhos, 
                    torch.ones(1,NSamp,device=device), 
                    [bvcopula.GaussianCopula_Likelihood().copula])
    copula_layers = []
    rho_cond = rho.clone()
    for i in range(Nvar-1):
        copula_layers.append([lin_gauss(rho_cond) for j in range(Nvar-1-i)])
        rho_cond = (rho_cond - rho_cond**2) / (1 - rho_cond**2)
    return copula_layers

In [8]:
copula_layers = const_rho_layers(rho0,Nvar)
vine = CVine(copula_layers,train_x,device=device)
#sample
y=vine.sample().cpu().numpy()
#transform samples
new_y = y.copy()
new_y += np.repeat(y.prod(axis=-1).reshape(NSamp,1),Nvar,axis=-1)**(1/Nvar)
transformed_y = (np.argsort(new_y.flatten()).argsort()/new_y.size).reshape(new_y.shape)
train_y = torch.tensor(transformed_y).float().to(device=device)

In [9]:
likelihoods =  [bvcopula.GaussianCopula_Likelihood(),
                bvcopula.GumbelCopula_Likelihood(rotation='180°'),
                bvcopula.GumbelCopula_Likelihood(rotation='0°')]
waic, model = bvcopula.infer(likelihoods,train_x,train_y,device=device)

In [10]:
get_R(model,copula)

[0.9792320898383986]